In [63]:
import numpy as np
from typing import List
import math
import pandas as pd

In [64]:
class Neuron:
    def __init__(self, layer, pos):
        self.layer = layer
        self.pos = pos
        self.value = 0
        self.weights: List[float] = []
        self.new_weights: List[float] = []
        self.prev_neurons: List[Neuron] = []
        self.next_neurons: List[Neuron] = []
        self.need_activation: bool = True

    def __repr__(self) -> str:
        if not self.prev_neurons:
            return f"""Neuron ({self.layer},{self.pos})
x -> {self.value}
"""
        elif not self.next_neurons:
            return f"""Neuron ({self.layer},{self.pos})
x -> {self.activate()}
Pesos -> {self.weights}
Bias -> {self.bias}
"""
        else:
            return f"""Neuron ({self.layer},{self.pos})
x -> {self.activate()}
Pesos -> {self.weights}
Bias -> {self.bias}
"""

    def activate(self):
        return 1 / (1 + math.exp(-self.value))

    def derivative(self):
        activate = self.activate()  # Calcular una sola vez
        return activate * (1 - activate)

    def calculate_value(self):
        for prev_neuron, weight in zip(self.prev_neurons, self.weights):
            if not prev_neuron.need_activation:
                self.value += prev_neuron.value * weight
            else:
                self.value += prev_neuron.activate() * weight
        self.value += self.bias

    def calculate_new_weight(self, label, learning_rate):
        if not self.next_neurons:  # Capa de salida
            self.delta = (self.activate() - label) * self.derivative()
        else:
            next_sum = 0
            for next_neuron in self.next_neurons:
                for prev_neuron, next_weight in zip(
                    next_neuron.prev_neurons, next_neuron.weights
                ):
                    if self == prev_neuron:
                        next_sum += next_neuron.delta * next_weight
                        break
            self.delta = next_sum * self.derivative()

        new_weights = [
            weight - (learning_rate * self.delta * prev_neuron.activate())
            for weight, prev_neuron in zip(self.weights, self.prev_neurons)
        ]
        self.new_weights = new_weights
        self.new_bias = self.bias - (learning_rate * self.delta)

In [96]:
class NeuralNetwork:
    def __init__(self, layers):  # nn = NeuralNetwork([5, 3, 2])
        self.layers: List[List[Neuron]] = []
        self.add_layers(layers)
        self.setup()

    def __repr__(self):
        string = ""
        for layer in self.layers:
            for neuron in layer:
                string += neuron.__repr__()
            string += "\n"
        return string

    def add_layers(self, layers):  # Agrega neuronas sin parametros
        for i, layer in enumerate(layers, start=1):
            self.layers.append([Neuron(i, j + 1) for j in range(layer)])

    def setup(self):  # Agrega listas de pesos, sesgos, neuronas previas y neuronas siguientes
        for i, layer in enumerate(self.layers):
            for neuron in layer:
                if i == 0:  # Capa de entrada
                    neuron.next_neurons = self.layers[i + 1]
                elif i == len(self.layers) - 1:  # Capas salida
                    neuron.weights = [0.10 * np.random.randn() for _ in range(len(self.layers[i - 1]))]
                    neuron.bias = 1
                    neuron.prev_neurons = self.layers[i - 1]
                else:  # Capa ocultas
                    neuron.weights = [0.10 * np.random.randn() for _ in range(len(self.layers[i - 1]))]
                    neuron.bias = 1
                    neuron.prev_neurons = self.layers[i - 1]
                    neuron.next_neurons = self.layers[i + 1]

    def predict(self, inputs):
        # outputs = []
        for i, layer in enumerate(self.layers):
            if i == 0:
                for neuron, x in zip(layer, inputs):
                    neuron.value = x
                    neuron.need_activation = False
            else:
                for neuron in layer:
                    neuron.calculate_value()
                    if i == len(self.layers) - 1:
                        output = neuron.activate()
        return output

    def update_weights(self, label, lr):
        for layer in reversed(self.layers[1:]):
            for neuron in layer:
                neuron.calculate_new_weight(label, lr)

        for layer in self.layers[1:]:
            for neuron in layer:
                neuron.weights = neuron.new_weights
                neuron.bias = neuron.new_bias

    def reset_values(self):
        for layer in self.layers:
            for neuron in layer:
                neuron.value = 0

    def train(self, X, y, n_epochs, lr=0.1):
        for epoch in range(1, n_epochs + 1):
            correct = 0
            error = 0
            for x, label in zip(X, y):
                output = self.predict(x)
                predicted = 0 if output < 0.5 else 1
                correct += int(predicted == label)
                error += (output - label) ** 2.0
                self.update_weights(label, lr)
                self.reset_values()
            accuracy = 100 * correct / len(X)
            loss = error / len(X)
            if epoch % 50 == 0:
                print("Epoch {}/{}, Loss: {:.8f}, Accuracy: {:.3f}".format(epoch, n_epochs, loss, accuracy))
        # print(f"Epochs {n_epochs}, Loss: {loss}, Accuracy: {accuracy}")

In [97]:
nn = NeuralNetwork([13, 5, 1])

In [87]:
# Lectura de datos desde dataset.csv
dataset_df = pd.read_csv("dataset.csv", usecols=[i for i in range(1, 15)])

In [88]:
dataset_df

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
0,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1
1,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1
2,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1
3,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1
4,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,0.00106,0.584,274404,0.932,0.002690,1,0.1290,-3.501,1,0.3330,74.976,4.0,0.211,0
2013,0.08770,0.894,182182,0.892,0.001670,1,0.0528,-2.663,1,0.1310,110.041,4.0,0.867,0
2014,0.00857,0.637,207200,0.935,0.003990,0,0.2140,-2.467,1,0.1070,150.082,4.0,0.470,0
2015,0.00164,0.557,185600,0.992,0.677000,1,0.0913,-2.735,1,0.1330,150.011,4.0,0.623,0


In [89]:
# Reordenamiento
dataset_df_copy = dataset_df.sample(frac=1).reset_index(drop=True)

In [90]:
# Datos de entrada y etiquetas
n_columns = len(dataset_df_copy.columns)
data_input = dataset_df_copy.iloc[:, 0 : n_columns - 1]
data_label = dataset_df_copy.iloc[:, n_columns - 1]


In [91]:
data_label

0       0
1       0
2       0
3       0
4       1
       ..
2012    0
2013    1
2014    0
2015    1
2016    0
Name: target, Length: 2017, dtype: int64

In [92]:
# Normalizacion
norm_cols = ["duration_ms", "key", "loudness", "tempo", "time_signature"]
for feature_name in data_input.columns:
    if feature_name in norm_cols:
        min_val = data_input[feature_name].max()
        max_val = data_input[feature_name].min()
        data_input[feature_name] = (data_input[feature_name] - min_val) / (max_val - min_val)

In [93]:
data_input

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.379000,0.534,0.787685,0.508,0.000000,0.818182,0.0867,0.206709,1,0.0322,0.381310,0.25,0.4070
1,0.765000,0.767,0.810617,0.445,0.000727,0.454545,0.0897,0.250686,1,0.0328,0.579733,0.25,0.3750
2,0.573000,0.498,0.813513,0.467,0.000000,0.909091,0.1030,0.212748,1,0.0300,0.592231,0.25,0.4020
3,0.168000,0.859,0.818356,0.756,0.000046,0.272727,0.1630,0.093108,0,0.0643,0.684065,0.25,0.9130
4,0.723000,0.688,0.584067,0.421,0.625000,-0.000000,0.0734,0.265569,0,0.0445,0.474311,0.25,0.6300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,0.000797,0.480,0.791610,0.826,0.000001,1.000000,0.1250,0.130985,1,0.0397,0.719249,0.25,0.7010
2013,0.074000,0.912,0.610849,0.781,0.762000,0.363636,0.3480,0.204666,0,0.0444,0.579412,0.25,0.8910
2014,0.912000,0.399,0.686881,0.307,0.623000,0.909091,0.1150,0.429186,1,0.0347,0.600815,0.25,0.0621
2015,0.031300,0.515,0.775055,0.902,0.000000,0.454545,0.0930,0.134858,0,0.3830,0.260929,-0.00,0.6170


In [94]:
# Division de datos
training_percent = 0.7
n_rows = len(dataset_df_copy.index)
n_train = round(training_percent * n_rows)
n_test = n_rows - n_train

X_train = data_input.iloc[0:n_train]
y_train = data_label.iloc[0:n_train]
X_test = data_input.iloc[n_test:]
y_test = data_label.iloc[n_test:]


0       1
1       0
2       1
3       1
4       1
       ..
1407    0
1408    1
1409    0
1410    1
1411    1
Name: target, Length: 1412, dtype: int64

In [98]:
n_epochs = 100
nn.train(X_train.values, y_train.values, n_epochs)

Epoch 50/100, Loss: 0.22460573, Accuracy: 63.952
Epoch 100/100, Loss: 0.21891022, Accuracy: 64.873


In [ ]:
xor_nn = NeuralNetwork([2, 2, 1])

X = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
]

y = [0, 1, 1, 0]

n_epochs = 100_000

nn.train(X, y, n_epochs)

3